In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score #confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint, uniform
from sklearn.impute import SimpleImputer

In [37]:
url = "http://dibresources.jcbose.ac.in/ssaha4/pulmopred/public/training.csv.txt"
ds = pd.read_csv(url)
ds = ds[~ds['Diagnosis'].isin(['DPLD', 'OSA', 'Sarcodiosis','Chest Pain'])]
ds = ds.drop(columns = ['Label'])
X_without_diagnosis = ds.drop('Diagnosis', axis = 1)

X = X_without_diagnosis
Y = ds['Diagnosis']

In [38]:
#X_without_diagnosis.head()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [40]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

rf = RandomForestClassifier()
hgbc = HistGradientBoostingClassifier()

rand_search_rf = RandomizedSearchCV(rf, param_distributions = param_dist, n_iter=5, cv=5)
rand_search_rf.fit(X_train_imputed, y_train)

best_rf = rand_search_rf.best_estimator_

KeyboardInterrupt: 

In [ ]:
#rf = RandomForestClassifier()
#rf = HistGradientBoostingClassifier()
rf.fit(X_train, y_train)

HistGradientBoostingClassifier()

In [ ]:
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print('Best hyperparameters:',  rand_search.best_params_)

Accuracy: 0.7649006622516556
